In [436]:
##
##Script to get List of categories available in Yelp that list restraurants in ranking
##
import pandas as pd
import numpy as np
#for creating geo coordinate combinations
import itertools  as it
from yelpapi import YelpAPI
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
%matplotlib inline

#### https://www.yelp.com/developers/v2/manage_api_keys refresh to generate new keys

In [437]:
api_key = 'SD5ybcet2QSwWXz2N7M5lw'
api_secret = 'EJBXJBAFgEka0br1ynT6HwWJVT8'
access_token = 'EESADq5kSfBL4XkbxzwJxsqhZIgQHbJs'
access_secret = 'JMU-eh8i0eVUUYxJcjelLkZEfTY'

#### activate yelpapi with personal keys

In [438]:
yelp_api = YelpAPI(api_key, api_secret, access_token, access_secret)

#### Invoke a presaved dataset, this is optional since the method creates new dataframe

In [459]:
East = pd.read_csv('../data/yelp_east.csv', parse_dates=True)
East = East.head(0)

## Create a method to grab yelp business with a keyword in a lat-long bound

In [477]:
#def getbiz(term, nw_latitude, nw_longitude,se_latitude, se_longitude):
#   #Create the first base dataframe
#  bounds = str(se_latitude)+','+str(se_longitude)+'|'+str(nw_latitude)+','+str(nw_longitude)
def getbiz(term, bounds):

    #Create the first base dataframe
    search_results_dict = yelp_api.search_query(term= term, bounds=bounds)
    search_results_df = {}
    
    if len(search_results_dict['businesses']) >0:
        search_results_df = json_normalize(search_results_dict['businesses'])
        offset = 0
        while offset<= 980:
            temp_dict = yelp_api.search_query(term= term, bounds=bounds, offset=offset)
            [n['id'] for n in search_results_dict['businesses']]
            temp_df = json_normalize(temp_dict['businesses'])
            #print offset
            
            if temp_df.shape[0] < 20:
                #YELPAPI gives upto 1000 results
                print 'Results exhausted at '+ str(temp_df.shape[0]+offset)+' results at offset '+str(offset)
                break
            #search_results_df = search_results_df.append(temp_df, ignore_index=True)
            search_results_df = pd.concat([search_results_df, temp_df], ignore_index=True)
            offset += 20

In [478]:

#Specify the keyword to search
term = 'bar'

#Specify the lat lon to specify bounds
nw_latitude=37.810298
nw_longitude=-122.522450
se_latitude=37.704093
se_longitude=-122.375803

#Specify how many rows and columns you wish to devide the geographical bounds into
block = 15

#Find latitude and longitude blocks coordinates
lat = np.arange(se_latitude, nw_latitude, (nw_latitude-se_latitude)/block)
lon = np.arange(se_longitude, nw_longitude, (nw_longitude-se_longitude)/block)

#build a block squared list of coordinates
import itertools
listoflist = lat,lon

#make all combinations of bounds required for calculation
comb  = list(itertools.product(*listoflist))

#sequence the list of coordinates to run north (east to west) to south (east to west)
comb = comb[::-1]

In [479]:
#Function to get 1 if a number is not a multiple of something 
import math
def rem(x,y):
    res = x % y
    return math.copysign(res,x)


In [480]:
#Create a list of all bounds to pass through getbiz() method

i=0
bounds = []
#print (block*(block-1)-1)
while i<=(block*(block-1)-1):
    if rem(i,block) != (block-1):
        bounds = bounds + [str(comb[i][0])+','+str(comb[i][1])+'|'+str(comb[i+1+block][0])+','+str(comb[i+1+block][1])]
        #print len(bounds)
        i += 1
    elif rem(i,block) == (block-1):
        i += 1
        #print 'this is where im at'+str(i)

In [481]:
print len(bounds)
print East.shape

196
(200, 45)


In [482]:
#Run getbiz() for every Bound

b = 191
#10, 24, 80
#64, 127, 196
while b <= (len(bounds)-1):
    getbiz(term,bounds[b])
    East = pd.concat([East,search_results_df], ignore_index=True)
    print b
    b+=1
East.shape

Results exhausted at 4 results at offset 0
191
Results exhausted at 1 results at offset 0
192
Results exhausted at 2 results at offset 0
193
Results exhausted at 1 results at offset 0
194
195


(400, 45)

In [483]:
#Export to csv
#East.to_csv('../data/yelp_east3.csv',encoding='utf-8')

In [484]:
len(bounds)

196

In [485]:
East.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'categories', u'deals',
       u'display_phone', u'gift_certificates', u'id', u'image_url',
       u'is_claimed', u'is_closed', u'location.address', u'location.city',
       u'location.coordinate.latitude', u'location.coordinate.longitude',
       u'location.country_code', u'location.cross_streets',
       u'location.display_address', u'location.geo_accuracy',
       u'location.neighborhoods', u'location.postal_code',
       u'location.state_code', u'location_address', u'location_city',
       u'location_coordinate_latitude', u'location_coordinate_longitude',
       u'location_country_code', u'location_cross_streets',
       u'location_display_address', u'location_geo_accuracy',
       u'location_neighborhoods', u'location_postal_code',
       u'location_state_code', u'menu_date_updated', u'menu_provider',
       u'mobile_url', u'name', u'phone', u'rating', u'rating_img_url',
       u'rating_img_url_large', u'rating_img_url_small', u'review_count

In [486]:
East.id.value_counts()

baiano-pizzeria-san-francisco                  20
crawstation-san-francisco-3                    20
sensi-luxury-nails-and-spa-san-francisco       20
roxies-market-and-deli-san-francisco           20
judah-ninth-avenue-liquors-san-francisco       20
snowbird-coffee-san-francisco                  20
nan-king-road-bistro-san-francisco             20
sunset-laundry-and-cleaner-san-francisco       20
donut-world-san-francisco                      20
cybelles-front-room-san-francisco              20
sunset-bakery-san-francisco                    20
arizmendi-bakery-san-francisco                 20
juns-beauty-salon-san-francisco                20
enjoy-vegetarian-restaurant-san-francisco-3    20
masala-dosa-san-francisco                      20
hing-wang-bakery-san-francisco                 20
meshi-sushi-san-francisco                      20
happy-donuts-san-francisco-5                   20
pierres-auto-body-san-francisco                20
holy-gelato-san-francisco                      20


In [389]:
East = East.drop_duplicates('id')

In [391]:
East.shape

(904, 45)

# Test Code

In [308]:
term = 'bar'
coordinates = bounds[127]
yelp_api.search_query(term= term, bounds=coordinates)
#search_results_df = {}

IndexError: string index out of range

In [309]:
term = 'bar'
bounds = bounds[13]
search_results_dict = yelp_api.search_query(term= term, bounds=bounds)


IndexError: string index out of range

In [199]:
search_results_df = json_normalize(search_results_dict['businesses'])

In [200]:
offset = 0
temp_dict = yelp_api.search_query(term= term, bounds=coordinates, offset=offset)
[n['id'] for n in search_results_dict['businesses']]

[u'crawstation-san-francisco-3',
 u'cybelles-front-room-san-francisco',
 u'donut-world-san-francisco',
 u'arizmendi-bakery-san-francisco',
 u'roxies-market-and-deli-san-francisco',
 u'meshi-sushi-san-francisco',
 u'baiano-pizzeria-san-francisco',
 u'enjoy-vegetarian-restaurant-san-francisco-3',
 u'snowbird-coffee-san-francisco',
 u'masala-dosa-san-francisco',
 u'nan-king-road-bistro-san-francisco',
 u'holy-gelato-san-francisco',
 u'hing-wang-bakery-san-francisco',
 u'sensi-luxury-nails-and-spa-san-francisco',
 u'juns-beauty-salon-san-francisco',
 u'sunset-laundry-and-cleaner-san-francisco',
 u'happy-donuts-san-francisco-5',
 u'sunset-bakery-san-francisco',
 u'judah-ninth-avenue-liquors-san-francisco',
 u'pierres-auto-body-san-francisco']

In [201]:
temp_df = json_normalize(temp_dict['businesses'])

In [202]:
temp_df.shape

(20, 30)

In [203]:
search_results_df = search_results_df.append(temp_df, ignore_index=True)

In [204]:
offset += 20
if temp_df.shape[0] < 20:
            #YELPAPI gives upto 1000 results
            print 'Results exhausted at '+ str(temp_df.shape[0])+' results at offset '+str(offset)
            break

SyntaxError: 'break' outside loop (<ipython-input-204-b2a52044b467>, line 5)

In [ ]:
if len(search_results_dict['businesses']) >=1:
    search_results_df = json_normalize(search_results_dict['businesses'])
    offset = 0
    while offset<= 980:
        temp_dict = yelp_api.search_query(term= term, bounds=coordinates, offset=offset)
        [n['id'] for n in search_results_dict['businesses']]
        temp_df = json_normalize(temp_dict['businesses'])
        search_results_df = search_results_df.append(temp_df, ignore_index=True)
        #print offset
        offset += 20
        if temp_df.shape[0] < 20:
            #YELPAPI gives upto 1000 results
            print 'Results exhausted at '+ str(temp_df.shape[0])+' results at offset '+str(offset)
            break
elif len(search_results_dict['businesses']) ==0:
    search_results_df = {}
search_results_df